In [ ]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import cv2

def get_grayscale_img(path, rescale_dim):
    img = cv2.imread(path)
    rescaled = cv2.resize(img, (rescale_dim, rescale_dim), cv2.INTER_LINEAR)
    gray = cv2.cvtColor(rescaled, cv2.COLOR_RGB2GRAY).astype('float')
    return gray

def save_img(img, num_id, desc):
    path = desc+num_id+".jpg"
    cv2.imwrite(path,img)
    return path
    
def save_grayscale_row(row, desc):
    path = row.path
    num_id = row.num_id
    gray = get_grayscale_img(path, 100)
    return save_img(gray, num_id, desc)

def save_grayscale(df):
    for index, row in df.iterrows():
        df.loc[index,'gray_path'] = save_grayscale_row(row,desc)
    return df

In [ ]:
df = cervix.data.make_base_df()
#small_df = df.sample(500,random_state=3)
#test_df = df.sample(30,random_state=4)

desc = 'gray_'
#test_df = save_grayscale(test_df)
#small_df = save_grayscale(small_df)
df = save_grayscale(df)

In [ ]:
test_df = cervix.test.make_test_df()
test_df = save_grayscale(df)

In [ ]:
def random_forest_transform(df, test=False):
    forest_df = pd.DataFrame()
    vecs = []
    types = []
    ids = []
    for _, row in df.iterrows():
        gray = cv2.imread(row.gray_path)
        vec = process_image(gray)
        if not test:
            cervix_type = row.type
            types.append(cervix_type[-1])
        else:
            ids.append(row.num_id)
        vecs.append(vec)
    if not test:
        return np.squeeze(np.array(vecs)), np.array(types)
    else:
        return np.squeeze(np.array(vecs)), np.array(ids)
                              
def process_image(img):
    """Normalize image and turn into array of one long column"""
    normalized = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX)
    vec = normalized.reshape(1, np.prod(normalized.shape))
    return vec / np.linalg.norm(vec)


In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC

vecs, types = random_forest_transform(df)
test_vecs, ids = random_forest_transform(test_df, True)
forest = RFC(n_jobs=2,n_estimators=50)

forest.fit(vecs, types)

In [ ]:
preds_probs = forest.predict_proba(test_vecs)

In [ ]:
import csv

def write_to_file(path, ids, preds):
    with open(path, 'w') as csv_f:
        writer = csv.writer(csv_f)
        writer.writerow(['image_name','Type_1','Type_2','Type_3'])
        for i,row in enumerate(preds_probs):
            path = [ids[i]+'.jpg']
            writer.writerow(path+list(row))

In [ ]:
write_to_file('output.csv', ids, preds_probs)